In [1]:
from src import dask_preperation as dp
from dask.distributed import Client, wait
from models import custom_RF as md
from load_dataset import custom_lakedepth as data_prep
from pprint import pprint
import time

In [2]:
dask = dp.Dask(1, 8)
c = Client(dask.cluster)
dask.set_client(c)
print(c)
c

<Client: 'tcp://127.0.0.1:35980' processes=1 threads=1, memory=540.49 GB>


Client Scheduler: tcp://127.0.0.1:35980 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 1 Cores: 1 Memory: 540.49 GB


In [3]:
data = data_prep.LakeDepth(42)
cv_train, cv_test, l_train, l_test = data.split(0.20)

 - from DATA: reading csv into GPU memory
 - from DATA: done reading csv into GPU memory
 - from DATA: dropped column: FID
 - from DATA: dropped column: Date


In [4]:
hyperparameters = {	   'N_ESTIMATORS' : 1000,
					   'SPLIT_ALGO' : 1,
					   'SPLIT_CRITERION' : 2,
					   'BOOTSTRAP' : True,
					   'BOOTSTRAP_FEATURES' : False,
					   'ROWS_SAMPLE' : 1.0,
					   'MAX_DEPTH' : 20,
					   'MAX_LEAVES' : -1,
					   'MAX_FEATURES' : 'auto',
					   'N_BINS' : 16,
					   'MIN_ROWS_PER_NODE' : 2,
					   'MIN_IMPURITY_DECREASE' : 0.0,
					   'ACCURACY_METRIC' : 'mean_ae', # 'mse' #'r2' # 'median_aw' # 
					   'QUANTILEPT' : False,
					   'SEED' :  42,
					   'VERBOSE' : False
					   }

In [5]:
pprint(hyperparameters)
rf = md.DaskCumlRF(hyperparameters)

{'ACCURACY_METRIC': 'mean_ae',
 'BOOTSTRAP': True,
 'BOOTSTRAP_FEATURES': False,
 'MAX_DEPTH': 20,
 'MAX_FEATURES': 'auto',
 'MAX_LEAVES': -1,
 'MIN_IMPURITY_DECREASE': 0.0,
 'MIN_ROWS_PER_NODE': 2,
 'N_BINS': 16,
 'N_ESTIMATORS': 1000,
 'QUANTILEPT': False,
 'ROWS_SAMPLE': 1.0,
 'SEED': 42,
 'SPLIT_ALGO': 1,
 'SPLIT_CRITERION': 2,
 'VERBOSE': False}


In [6]:
cv_dt, l_dt = dask.distribute(cv_train, l_train)
cv_dte, l_dte = dask.distribute(cv_test, l_test)

In [7]:
st = time.time()
rf.train(cv_dt, l_dt)
et = time.time()

In [8]:
%%time
cv_test_pd = cv_test.to_pandas()
cuml_y_pred = rf.model.predict(cv_test_pd)

ValueError: Cannot create cython.array from NULL pointer